In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect("host=127.0.0.1 port=5433 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

## Query to look at the top five artists with the most song plays for the entire history of the data (currently returns nothing because none of the songs played are referenced in the artists or songs tables):

In [3]:
cur.execute(
    """
    SELECT
        songplays.artist_id,
        artists.name,
        COUNT(DISTINCT(songplays.songplay_id))
    FROM
        songplays
    INNER JOIN artists USING (artist_id)
    GROUP BY
        songplays.artist_id,
        artists.name
    """
)
results = cur.fetchall()
results

[]

## Query to look at songplays by hour of the day:

In [4]:
cur.execute(
    """
    SELECT
        time.hour,
        COUNT(DISTINCT(songplays.songplay_id))
    FROM
        songplays
    INNER JOIN time USING (start_time)
    GROUP BY time.hour
    """
)
results = cur.fetchall()
results

[(0.0, 155),
 (1.0, 154),
 (2.0, 117),
 (3.0, 109),
 (4.0, 136),
 (5.0, 162),
 (6.0, 183),
 (7.0, 179),
 (8.0, 207),
 (9.0, 270),
 (10.0, 312),
 (11.0, 336),
 (12.0, 308),
 (13.0, 324),
 (14.0, 432),
 (15.0, 477),
 (16.0, 542),
 (17.0, 494),
 (18.0, 498),
 (19.0, 367),
 (20.0, 360),
 (21.0, 280),
 (22.0, 217),
 (23.0, 201)]

In [5]:
hourly_data = list(zip(*results))
hourly_data

[(0.0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0),
 (155,
  154,
  117,
  109,
  136,
  162,
  183,
  179,
  207,
  270,
  312,
  336,
  308,
  324,
  432,
  477,
  542,
  494,
  498,
  367,
  360,
  280,
  217,
  201)]

In [6]:
# Turn it into a graph using plotly
import plotly.express as px
fig = px.bar(x=hourly_data[0],y=hourly_data[1],
             title="Hourly Songplays (All Time)")
fig.update_xaxes(title="Hour")
fig.update_xaxes(tickvals=list(range(0,23)))
fig.update_yaxes(title="Songplays")
fig.show()

## Query to look at the top 10 users

In [7]:
cur.execute(
    """
    SELECT
        users.first_name || ' ' || users.last_name AS fullname,
        COUNT(DISTINCT(songplays.songplay_id)) AS play_count
    FROM
        songplays
    INNER JOIN users USING (user_id)
    GROUP BY
        fullname
    ORDER BY
        play_count DESC
    LIMIT 10
    """
)
results = cur.fetchall()
results

[('Chloe Cuevas', 689),
 ('Tegan Levine', 665),
 ('Kate Harrell', 557),
 ('Lily Koch', 463),
 ('Aleena Kirby', 397),
 ('Jacqueline Lynch', 346),
 ('Layla Griffin', 321),
 ('Jacob Klein', 289),
 ('Mohammad Rodriguez', 270),
 ('Matthew Jones', 248)]

## Query to look at the songplays by gender

In [8]:
cur.execute(
    """
    SELECT
        users.gender,
        COUNT(DISTINCT(songplays.songplay_id)) AS play_count
    FROM
        songplays
    INNER JOIN users USING (user_id)
    GROUP BY
        gender
    """
)
results = cur.fetchall()
results

[('F', 4887), ('M', 1933)]

In [9]:
gender_data = list(zip(*results))
gender_data

[('F', 'M'), (4887, 1933)]

In [10]:
# Turn it into a graph using plotly
fig = px.bar(x=gender_data[0],y=gender_data[1],
             title="Songplays (All Time) by Gender")
fig.update_xaxes(title="Gender")
fig.update_xaxes(
    ticktext=["Female", "Male"],
    tickvals=["F","M"]   
)
fig.update_yaxes(title="Songplays")
fig.show()

## Query to see the timefame of the songplays -> 11/1/2018 - 11/30/2018

In [11]:
cur.execute(
    """
    SELECT
        MAX(DATE(songplays.start_time)) AS max_date,
        MIN(DATE(songplays.start_time)) AS min_date
    FROM
        songplays
    """
)
results = cur.fetchall()
results

[(datetime.date(2018, 11, 30), datetime.date(2018, 11, 1))]

## Query to find out where the artists are located

In [12]:
cur.execute(
    """
    SELECT
        artists.name,
        artists.latitude,
        artists.longitude
    FROM
        artists
    WHERE
        artists.latitude IS NOT NULL
        AND artists.longitude IS NOT NULL
    """
)
results = cur.fetchall()
results

[('The Box Tops', 35.14968, -90.04892),
 ('JennyAnyKind', 35.21962, -80.01955),
 ('Blue Rodeo', 43.64856, -79.38533),
 ('Richard Souther', 37.16793, -95.84502),
 ('Jimmy Wakely', 34.31109, -94.02978),
 ('Alice Stuart', 38.8991, -77.029),
 ('Elena', 49.80388, 15.47491),
 ('The Dillinger Escape Plan', 40.82624, -74.47995),
 ('SUE THOMPSON', 37.83721, -94.35868),
 ('Tim Wilson', 32.67828, -83.22295),
 ('Willie Bobo', 40.79195, -73.94512),
 ('Faye Adams', 40.73197, -74.17418),
 ('Terry Callier', 41.88415, -87.63241),
 ('John Wesley', 27.94017, -82.32547),
 ('The Shangri-Las', 40.7038, -73.83168),
 ('Billie Jo Spears', 30.08615, -94.10158),
 ('Backstreet Boys', 28.53823, -81.37739),
 ('Tiny Tim', 40.71455, -74.00712),
 ('David Martin', 37.77916, -122.42005),
 ('John Davis', 40.99471, -77.60454),
 ('Trafik', 39.49974, -111.54732),
 ('Nick Ingman;Gavyn Wright', 51.50632, -0.12714),
 ('Steel Rain', 37.77916, -122.42005),
 ('Gloriana', 36.16778, -86.77836),
 ('Bombay Rockers', 56.27609, 9.51695

In [13]:
artist_loc = list(zip(*results))
artist_loc

[('The Box Tops',
  'JennyAnyKind',
  'Blue Rodeo',
  'Richard Souther',
  'Jimmy Wakely',
  'Alice Stuart',
  'Elena',
  'The Dillinger Escape Plan',
  'SUE THOMPSON',
  'Tim Wilson',
  'Willie Bobo',
  'Faye Adams',
  'Terry Callier',
  'John Wesley',
  'The Shangri-Las',
  'Billie Jo Spears',
  'Backstreet Boys',
  'Tiny Tim',
  'David Martin',
  'John Davis',
  'Trafik',
  'Nick Ingman;Gavyn Wright',
  'Steel Rain',
  'Gloriana',
  'Bombay Rockers',
  'Jinx',
  'Danilo Perez',
  'King Curtis',
  'Sophie B. Hawkins',
  'Lupe Fiasco',
  'Bitter End'),
 (35.14968,
  35.21962,
  43.64856,
  37.16793,
  34.31109,
  38.8991,
  49.80388,
  40.82624,
  37.83721,
  32.67828,
  40.79195,
  40.73197,
  41.88415,
  27.94017,
  40.7038,
  30.08615,
  28.53823,
  40.71455,
  37.77916,
  40.99471,
  39.49974,
  51.50632,
  37.77916,
  36.16778,
  56.27609,
  45.80726,
  8.4177,
  32.74863,
  40.79086,
  41.88415,
  -13.442),
 (-90.04892,
  -80.01955,
  -79.38533,
  -95.84502,
  -94.02978,
  -77.0

In [15]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scattergeo(
    text=artist_loc[0],
    lat=artist_loc[1],
    lon=artist_loc[2],
    mode='markers'
))
fig.update_layout(
    title='Artist Locations'
)

fig.show()